## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-16-22-41-00 +0000,wsj,Fed’s Williams Says Rate Stance Remains ‘Entir...,https://www.wsj.com/economy/central-banking/fe...
1,2025-07-16-22-29-40 +0000,bbc,Sirens and evacuations as Taipei rehearses to ...,https://www.bbc.com/news/articles/cy0w6wrlld0o
2,2025-07-16-22-15-06 +0000,cbc,First Nations arrive with some optimism — but ...,https://www.cbc.ca/news/indigenous/carney-firs...
3,2025-07-16-22-03-22 +0000,bbc,Fire engulfs main stage of Tomorrowland festiv...,https://www.bbc.com/news/articles/cpqn3w0w7x5o
4,2025-07-16-21-55-04 +0000,bbc,'It's just better!' Trump says Coca-Cola to ch...,https://www.bbc.com/news/articles/czxe59zl8qzo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2360/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
36,trump,12
115,canada,4
12,china,4
151,israel,3
50,hit,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
15,2025-07-16-19-56-53 +0000,nyt,Carney Moves to Reduce Canada’s Chinese Steel ...,https://www.nytimes.com/2025/07/16/world/canad...,32
64,2025-07-16-00-00-00 +0000,wsj,"President Trump wants tariffs, the higher the ...",https://www.wsj.com/economy/trade/forget-taco-...,32
12,2025-07-16-20-44-00 +0000,wsj,The nation’s biggest bankers are speaking out ...,https://www.wsj.com/economy/central-banking/ba...,32
19,2025-07-16-19-15-59 +0000,bbc,Canada curbs steel imports to shield domestic ...,https://www.bbc.com/news/articles/cy5wv7rzvk0o,31
10,2025-07-16-20-53-24 +0000,bbc,Trump discussed firing Fed boss but 'highly un...,https://www.bbc.com/news/articles/c4geyrdprwjo,26


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
15,32,2025-07-16-19-56-53 +0000,nyt,Carney Moves to Reduce Canada’s Chinese Steel ...,https://www.nytimes.com/2025/07/16/world/canad...
12,20,2025-07-16-20-44-00 +0000,wsj,The nation’s biggest bankers are speaking out ...,https://www.wsj.com/economy/central-banking/ba...
5,19,2025-07-16-21-48-16 +0000,bbc,US says 'specific steps' agreed to end Syria v...,https://www.bbc.com/news/articles/cp90l77187zo
44,15,2025-07-16-10-35-23 +0000,nyt,South African AIDS Activist Pushes for H.I.V. ...,https://www.nytimes.com/2025/07/14/health/sout...
10,14,2025-07-16-20-53-24 +0000,bbc,Trump discussed firing Fed boss but 'highly un...,https://www.bbc.com/news/articles/c4geyrdprwjo
64,14,2025-07-16-00-00-00 +0000,wsj,"President Trump wants tariffs, the higher the ...",https://www.wsj.com/economy/trade/forget-taco-...
42,14,2025-07-16-12-20-00 +0000,world,Nvidia CEO Lavishes Praise on China in Beijing...,https://www.wsj.com/world/asia/nvidia-ceo-lavi...
22,13,2025-07-16-17-40-33 +0000,bbc,Faisal Islam: Surprise inflation jump complica...,https://www.bbc.com/news/articles/c70xy9nv7p7o
34,13,2025-07-16-14-36-00 +0000,wsj,EU Exports to U.S. Cool as Trans-Atlantic Trad...,https://www.wsj.com/economy/trade/eu-exports-t...
3,12,2025-07-16-22-03-22 +0000,bbc,Fire engulfs main stage of Tomorrowland festiv...,https://www.bbc.com/news/articles/cpqn3w0w7x5o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
